# Busca no Espaço de Configuração - Hiperparameter Tunning

# Índice:
   * [Descrição](#description)
      * [Tipos de Busca](#searchspace)
      * [Objetivos](#goals)
   * [Imports](#imports)
   * [Extração e Transformação](#extraction)
   * [Parâmetros Alvos da Busca](#parameters)
      * [Introdução a Modelos de Árvores de Decisão](#decisiontree)
      * [Parâmetros do SKLearn](#sklearnparams)
   * [Busca em Grade](#grid)      
      * [Treinamento](#training) 
      * [Validação](#validation)
      * [Log no MLFlow](#mlflow) 
         * [Métricas e Parâmetros](#logmetrics)
         * [Artefato com os Resultados](#logartifact)
   * [Busca Aleatorizada](#randomized)
      * [Treinamento](#randomizedTraining)
      * [Validação](#randomizedValidation)
      * [Log no MLFlow](#randomizedMLFlow)

## Descrição <a class="anchor" id="description"></a>

Esse notebook implementa a busca por híperparâmetros ótimos para o modelo de Árvore de Decisão. Esse modelo foi escolhido por ter um espaço de parâmetros com múltiplas dimensões e por possuir métodos que permitem analisar a tomada de decisão do modelo treinado.

### Tipos de Busca <a class="anchor" id="searchspace"></a>

São implementadas duas estratégias de busca nos híper-parâmetros: a busca randomizada e a busca em grade.

Na busca randomizada são passados uma distribuição de probabilididades para cada parâmetro e o número de iterações. O algoritmo irá gerar valores aleatórios de acordo com a distribuição para cada iteração.

Na busca em grade são passados um conjunto de valores para cada parâmetro e o algoritmo executará todas as combinações. 

Em ambos os casos, são passadas métricas de desempenho e uma delas deve ser determinada como o _score_ a ser utilizado na seleção do melhor modelo. Por motivos de otimização, a biblioteca _Sklearn_ utiliza funções de métricas que são maiores quanto melhor o modelo. Com isso as funções de erro absoluto (funções as quais um valor menor indica um modelo melhor) são multiplicadas por -1, para que cresçam conforme o desempenho do modelo melhora.

### Objetivos  <a class="anchor" id="goals"></a>

* desenvolver funções para a busca nos híperparâmetros (incluindo a validação e log no MLFlow),
* realizar a análise do desempenho do modelo no espaço dos parâmetros,
* entender como o modelo de arvore de decisão se aplica aos dados do problema

## Imports  <a class="anchor" id="imports"></a>

In [1]:
from time import time
import os
import sys
from tabnanny import verbose
import pandas as pd
import numpy as np
import yaml
from itertools import chain, combinations
import datetime
import tempfile

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.utils import shuffle

import mlflow
import mlflow.sklearn
from mlflow import artifacts
from mlflow.tracking import MlflowClient
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID


In [ ]:
#%pip install boto3

## Extração e Transformação <a class="anchor" id="extraction"></a>

A busca nos híper-parâmetros é feita no conjunto de treino, e o algoritmo de validação cruzada separa uma porção desse conjunto a cada iteração para usar como teste. Ao final, se o objetivo for produzir um modelo treinado, é utilizado um conjunto de validação para aferir o desempenho do melhor estimador encontrado na busca.

Abaixo são separados os conjuntos de treino e validação

In [2]:
def read_data(url):
    return pd.read_csv(os.path.abspath(url))

In [3]:
df = read_data("../extracao/datanov2.csv")

In [4]:
test_split = 0.3
random_state = 10

In [5]:
def getXy(df):
    r_state = 15
    df = shuffle(df, random_state=r_state)
    X = df.iloc[:,:-1]
    y = df.iloc[:,-1]
    return X, y

X, y = getXy(df)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.3, random_state=random_state
)


## Parâmetros Alvos da Busca<a class="anchor" id="parameters"></a>

### Introdução a Modelos de Árvores de Decisão <a class="anchor" id="decisiontree"></a>

Uma breve descrição do modelo de árvore de decisão é necessária para proceder na busca por híper-parâmetros, com o intuito de se entender melhor os argumentos os quais se deseja maximizar.

Árvores de decisão são grafos acíclicos que podem ser usados para realizar decisões. Em cada nó interno da árvore possui um índice de uma das características e um limiar. Um input que seja analisado nesse nó seguirá na subárvore à esquerda desse nó se o valor da característica indexada no nó for menor que o limiar, e seguirá na subárvore da direita caso contrário. Nas folhas é feita a previsão, que é constante para todos os nós que terminarem de percorrer a árvore naquele nó. Dessa forma, uma árvore é uma aproximação  constante em trechos do domínio.

### Parâmetros do SKLearn<a class="anchor" id="sklearnparams"></a>:
   * _max_depth_ : limita a altura máxima da árvore. Conforme a altura cresce, o modelo fica mais complexo, isso pode aumentar a performace, mas também pode levar a _over fitting_.
   * _max_features_ : Determina o número máximo de características a serem utilizadas nos nós internos da árvore para dividir as amostras.
   * _min_samples_split_ : número mínimo de amostras para dividir uma folha, transformando-a em um nó interno com duas folhas.
   * _min_samples_leaf_ : número mínimo de amostras permitidas em uma folha.
   
O parâmetro _max_depth_ é o príncipal parâmetro para regular o desempenho e o _over fitting_, mas note que os dois últimos parâmetros podem também ter influência na altura final da árvore, se os valores forem sufientemente altos para o conjunto de dados.



In [30]:
range_grid = {
    "max_depth": [ 1, 20, 2],
    "min_samples_split": [ 5, 600, 30],
    "min_samples_leaf": [ 5, 600, 30]
}

In [31]:
param_grid = {}
for k, v in range_grid.items():
        param_grid[k] = range(v[0], v[1], v[2])

In [32]:
param_grid

{'max_depth': range(1, 20, 2),
 'min_samples_split': range(5, 600, 30),
 'min_samples_leaf': range(5, 600, 30)}

## Métricas<a class="anchor" id="metrics"></a>

As métricas utilizadas são as mesmas dos últimos modelos: raíz do erro quadrático médio, erro absoluto médio e coeficiente de determinação (r2). Essa última é utilizada como _score_ pela busca em grade para determinar o melhor modelo, que será retreinado utilizando todo conjunto de treino, após a etapa de busca dos melhores parâmetros. Esse retreinamento (refit) permite tanto a validação do modelo na parcela dos dados não utilizada na busca quanto o log do modelo no MLFlow, que poderá ser baixado para uma análise posterior ou mesmo para um possível _deploy_.


In [33]:
_scoring = [
    "r2",
    "neg_mean_absolute_error",
    "neg_root_mean_squared_error",
    ]

## Treinamento <a class="anchor" id="training"></a>

Nesta etapa é treinado o modelo de Árvore de Decisão. O log do modelo no MLFlow é feito logo após o treinamento, por conta de uma limitação da API.

In [20]:
def connectMLFlow(MLFlowAddr):
    client = MlflowClient(tracking_uri=MLFlowAddr)
    mlflow.set_tracking_uri(MLFlowAddr)
    return client

In [34]:
client = connectMLFlow("http://172.27.0.1:5000")

In [35]:
experiment_name = "HiperParameter Search"
try:
    experiment_id = client.create_experiment(experiment_name)
except:
    experiment_id = client.get_experiment_by_name(experiment_name).experiment_id
    
experiment = mlflow.set_experiment(experiment_name)
run_name = "DecisionTree GridSearch"

In [36]:
skmodel = DecisionTreeRegressor(random_state=random_state)
gridSearchModel = GridSearchCV(
    skmodel, param_grid, scoring=_scoring, refit="neg_mean_absolute_error", verbose=1,return_train_score=True
)

In [37]:
grid_start = time()
with mlflow.start_run(run_name=run_name) as run:
    run_id = run.info.run_id
    gridSearchModel.fit(X_train, y_train)
    mlflow.sklearn.log_model(gridSearchModel.best_estimator_,"model/")
    mlflow.end_run()
grid_end = time()
grid_run_time = grid_end - grid_start

print ("Tempo para rodar essa célula: {}m {:.3f}s".format(int(grid_run_time/60), grid_run_time%60))

Fitting 5 folds for each of 4000 candidates, totalling 20000 fits
[CV] END max_depth=1, min_samples_leaf=5, min_samples_split=5; total time=   0.6s
[CV] END max_depth=1, min_samples_leaf=5, min_samples_split=5; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=5, min_samples_split=5; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=5, min_samples_split=5; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=5, min_samples_split=5; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=5, min_samples_split=35; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=5, min_samples_split=35; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=5, min_samples_split=35; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=5, min_samples_split=35; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=5, min_samples_split=35; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=5, min_samples_split=65; total time=   0.0s
[CV] END max_depth=1, min_

[CV] END max_depth=1, min_samples_leaf=35, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=35, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=35, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=35, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=35, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=35, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=35, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=35, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=35, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=35, min_samples_split=155; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=35, min_samples_split=155; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=35, min_samples_split=155; tot

[CV] END max_depth=1, min_samples_leaf=65, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=65, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=65, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=65, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=65, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=65, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=65, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=65, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=65, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=65, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=65, min_samples_split=155; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=65, min_samples_split=155; tota

[CV] END max_depth=1, min_samples_leaf=95, min_samples_split=65; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=95, min_samples_split=65; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=95, min_samples_split=65; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=95, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=95, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=95, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=95, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=95, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=95, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=95, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=95, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=95, min_samples_split=125; total t

[CV] END max_depth=1, min_samples_leaf=125, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=125, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=125, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=125, min_samples_split=95; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=125, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=125, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=125, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=125, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=125, min_samples_split=125; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=125, min_samples_split=155; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=125, min_samples_split=155; total time=   0.0s
[CV] END max_depth=1, min_samples_leaf=125, min_samples_sp

...

[CV] END max_depth=19, min_samples_leaf=455, min_samples_split=305; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=455, min_samples_split=305; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=455, min_samples_split=305; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=455, min_samples_split=305; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=455, min_samples_split=335; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=455, min_samples_split=335; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=455, min_samples_split=335; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=455, min_samples_split=335; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=455, min_samples_split=335; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=455, min_samples_split=365; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=455, min_samples_split=365; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=455

[CV] END max_depth=19, min_samples_leaf=485, min_samples_split=335; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=485, min_samples_split=335; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=485, min_samples_split=335; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=485, min_samples_split=335; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=485, min_samples_split=365; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=485, min_samples_split=365; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=485, min_samples_split=365; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=485, min_samples_split=365; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=485, min_samples_split=365; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=485, min_samples_split=395; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=485, min_samples_split=395; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=485

[CV] END max_depth=19, min_samples_leaf=515, min_samples_split=395; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=515, min_samples_split=395; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=515, min_samples_split=395; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=515, min_samples_split=395; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=515, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=515, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=515, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=515, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=515, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=515, min_samples_split=455; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=515, min_samples_split=455; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=515

[CV] END max_depth=19, min_samples_leaf=545, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=545, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=545, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=545, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=545, min_samples_split=455; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=545, min_samples_split=455; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=545, min_samples_split=455; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=545, min_samples_split=455; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=545, min_samples_split=455; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=545, min_samples_split=485; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=545, min_samples_split=485; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=545

[CV] END max_depth=19, min_samples_leaf=575, min_samples_split=395; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=575, min_samples_split=395; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=575, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=575, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=575, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=575, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=575, min_samples_split=425; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=575, min_samples_split=455; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=575, min_samples_split=455; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=575, min_samples_split=455; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=575, min_samples_split=455; total time=   0.0s
[CV] END max_depth=19, min_samples_leaf=575

## Validação <a class="anchor" id="validation"></a>

In [25]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [38]:
gridPredictionVal = gridSearchModel.predict(X_val)
gridPredictionTrain = gridSearchModel.predict(X_train)

rmse, mae, r2 = eval_metrics(y_train, gridPredictionTrain)
print (f"Desempenho no conjunto de Treino\nrmse:{rmse}    mae:{mae}    r2:{r2}")

rmse, mae, r2 = eval_metrics(y_val, gridPredictionVal)
print (f"Desempenho no conjunto de Testes (Validação)\nrmse:{rmse}    mae:{mae}    r2:{r2}")


Desempenho no conjunto de Treino
rmse:12.522044099688351    mae:9.710029222228924    r2:0.3509999859058073
Desempenho no conjunto de Testes (Validação)
rmse:13.09465127294294    mae:10.139468613569441    r2:0.25446618941123134


In [39]:
gridSearchModel.best_params_

{'max_depth': 9, 'min_samples_leaf': 35, 'min_samples_split': 125}

## Log no MLFlow <a class="anchor" id="mlflow"></a>

In [28]:
def logCVResultsCSV(cv_results, run_id, mlclient ):

    tempdir = tempfile.TemporaryDirectory(suffix=None, prefix=None, dir=None)
    tmpname = tempdir.name
    
    filename = "cv_results.csv"
    csv = os.path.join(tmpname, filename)
    
    pd.DataFrame(cv_results).to_csv(csv, index=False)
    
    mlclient.log_artifact(run_id,csv, "cv_results")
    tempdir.cleanup()


In [40]:
tags = {"search_type":"GridSearchCV",
        "model_random_state":str(random_state),
        "test_size":str(test_split),
        "test_random_state":str(random_state)      
       }
tags = [mlflow.entities.RunTag(k,v) for k,v in tags.items()]

search_params = {"param_grid":str(param_grid),
                 "scoring":str(_scoring),
                 "refit":"neg_mean_absolute_error",
                 "estimator":"DecisionTreeRegressor",
                }
search_params = [mlflow.entities.Param(k,v) for k,v in search_params.items()]

metrics = {"rmse":rmse, "mae":mae, "r2":r2}
now = int(time())
metrics = [mlflow.entities.Metric(k,v,now,1) for k,v in metrics.items()]
client.log_batch(run_id, metrics = metrics, params=search_params,tags = tags)
    
logCVResultsCSV(gridSearchModel.cv_results_, run_id, client)


## Busca Aleatorizada<a class="anchor" id="randomized"></a>

Aqui é feita uma busca aleatorizada com os mesmos parâmetros da busca em grade. Serão geradas 500 amostras, o que permite buscar o mesmo espaço de parâmetros de forma mais rápida, já que a busca por grade gerou 20000 diferentes combinações de parâmetros. No notebook seguinte está uma análise comparativa dos resultados, mas já podemos observar o desempenho na etapa de validação.

Como dito anteriormente, na busca aleatorizada são geradas amostras dos parâmetros com base em uma distribuição passada para cada um. Neste caso a distribuição foi constante para todos os parâmetros.

### Treinamento <a class="anchor" id="randomizedTraining"></a>

In [43]:
run_name = "DecisionTree RandomSearch"

In [15]:
n_iter = 500
randSearchModel = RandomizedSearchCV(
    skmodel, param_grid, scoring=_scoring, refit="neg_mean_absolute_error", verbose=1, n_iter=n_iter, random_state=random_state
)

In [16]:
rand_start = time()
with mlflow.start_run(run_name=run_name) as run:
    run_id = run.info.run_id
    randSearchModel.fit(X_train, y_train)
    mlflow.sklearn.log_model(randSearchModel.best_estimator_,"model/")
    mlflow.end_run()
rand_end = time()
rand_runtime = rand_end - rand_start
print ("Tempo para rodar essa célula: {}m {:.3f}s".format(int(rand_runtime/60), rand_runtime%60))

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


KeyboardInterrupt: 

### Validação<a class="anchor" id="randomizedValidation"></a>

In [47]:
randPredictVal = randSearchModel.predict(X_val)
randPredictTrain = randSearchModel.predict(X_train)

rmse, mae, r2 = eval_metrics(y_train, randPredictTrain)
print (f"Desempenho no conjunto de Treino\nrmse:{rmse}    mae:{mae}    r2:{r2}")

rmse, mae, r2 = eval_metrics(y_val, randPredictVal)
print (f"\nDesempenho no conjunto de Testes (Validação)\nrmse:{rmse}    mae:{mae}    r2:{r2}")
print ("\nTempo de busca: {:.3f}s".format(rand_end-rand_start))

Desempenho no conjunto de Treino
rmse:12.432063053099698    mae:9.635086784446763    r2:0.3602936575478374

Desempenho no conjunto de Testes (Validação)
rmse:13.118030155238092    mae:10.154019710466764    r2:0.2518016959212602

Tempo de busca: 31.441s


Vemos que o modelo encontrado apresentou um desempenho similar do que o da busca por grade, mesmo com menos tempo de treino. Abaixo estão os valores dos parâmetros do modelo de melhor desempenho na busca e as previsões feitas pelo modelo comparadas com os valores verdadeiros:

In [46]:
randSearchModel.best_params_

{'min_samples_split': 125,
 'min_samples_leaf': 35,
 'max_features': 6,
 'max_depth': 11}

### Log no MLFlow<a class="anchor" id="randomizedMLFlow"></a>

In [49]:
tags = {"search_type":"RandomSearchCV", 
        "model_random_state":str(random_state),
        "test_size":str(test_split),
        "test_random_state":str(random_state)
       }
tags = [mlflow.entities.RunTag(k,v) for k,v in tags.items()]

search_params = {"param_distributions":str(param_grid),
                 "scoring":str(_scoring),
                 "refit":"neg_mean_absolute_error",
                 "n_iter":str(n_iter),
                 "estimator":"DecisionTreeRegressor",
                "random_state":str(random_state)
                }
search_params = [mlflow.entities.Param(k,v) for k,v in search_params.items()]

metrics = {"rmse":rmse, "mae":mae, "r2":r2}
now = int(time()*1000)
metrics = [mlflow.entities.Metric(k,v,now,1) for k,v in metrics.items()]
client.log_batch(run_id, metrics = metrics, params=search_params,tags = tags)
    
logCVResultsCSV(randSearchModel.cv_results_, run_id, client)


### TODO

 * Tentar mudar o log do mlflow para ficar separado do treino. Usar start_run(run_id) com run_id vindo de create_run.
 * Checar os nomes. Passar camelCase para under_score.
 * Revisar a descrição.
 * Logar no Mlflow o train/test split e random state associado.